In [39]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV,Lasso,LinearRegression
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,scale,MaxAbsScaler,RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [5]:
df_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv')
c_int=df_train.select_dtypes(include=['int64']).apply(pd.to_numeric,downcast='unsigned')
c_float=df_train.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')

o_df=pd.concat([c_int,c_float],axis=1)
column_type=dict(o_df.dtypes.copy())

del o_df,c_int,c_float,df_train

In [6]:
data=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

# data_test=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',
#                       dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [12]:
data[(data.avg_online_minutes >=12) & (data.pay_price < data.prediction_pay_price)].shape

(14939, 109)

In [16]:
data[(data.avg_online_minutes >=12)].shape

(275717, 109)

In [161]:
data[(data['prediction_pay_price'] > data['pay_price']) & data['pay_price'] > 0.99] ['avg_online_minutes'].describe()

count    11309.000000
mean       245.507996
std        180.424377
min          0.666667
25%        109.666664
50%        200.500000
75%        339.833344
max       1674.666626
Name: avg_online_minutes, dtype: float64

In [17]:
data_train = data[data['avg_online_minutes'] >= 12].copy()

In [18]:
data_train.shape

(275717, 109)

In [19]:
data_train = data_train.drop(['user_id','register_time'],axis=1)

In [20]:
x_train = data_train.loc[:, data_train.columns != 'prediction_pay_price']
y_train = data_train.loc[:, data_train.columns == 'prediction_pay_price']

In [28]:
train_7pay=data[data['pay_price']>0].copy()
train_7pay=train_7pay.drop(['register_time'],axis=1)
# test_7pay=data_test2[data_test2['pay_price']>0].copy()

# train_7pay2=data_train2[data_train2['pay_price']>0].copy()
# test_7pay2=data_test2[data_test2['pay_price']>0].copy()
train_7pay_y=train_7pay.prediction_pay_price
train_7pay_X=train_7pay.drop(labels=['prediction_pay_price','user_id'],axis=1)
train_7pay_user_id=train_7pay.user_id
# test_7pay_X=test_7pay.drop(labels=['user_id'],axis=1)
# test_7pay_user_id=test_7pay.user_id

drop_col=[i for i in train_7pay.columns.values if 'prediction' in  i]
drop_col.append('user_id')
X=train_7pay.drop(labels=drop_col,axis=1)
y=train_7pay.prediction_pay_price
# X=train_7pay_X
y=train_7pay_y
X.replace([np.inf,-np.inf],np.nan,inplace=True)
X.fillna(0.0,inplace=True)

X['user_id']=train_7pay_user_id

X1,val_X1,y1,val_y1=train_test_split(X,y,test_size=0.2,random_state=17)
X2=X1.drop(labels='user_id',axis=1)
val_X2=val_X1.drop(labels='user_id',axis=1)
X2_user_id=X1.user_id
val_X2_user_id=val_X1.user_id

X.drop(labels='user_id',axis=1,inplace=True)

702488      883635
1536614    1990570
11227        12819
2034737    2895886
1661662    2243560
2248171    3148938
2229203    3096565
636882      816388
585327      763974
344818      387714
1838324    2586630
1787252    2409960
1989379    2814820
750539      932347
1907104    2728834
1221125    1535539
1933460    2755190
1942391    2764121
1192874    1507288
842856     1063496
99690       125950
1820601    2568907
1525347    1960296
730313      911460
287318      324317
1174092    1486891
1428029    1859175
1330133    1741863
38624        60243
1360344    1789589
            ...   
1121648    1391536
1627176    2207536
1025715    1272961
1067768    1337656
693223      873204
844386     1065026
236191      271402
1039972    1305330
2241452    3142219
1304364    1714428
161818      196181
278672      315671
149627      182309
1912980    2734710
2276241    3178765
2083573    2946361
173729      208092
1159936    1454010
97594       123854
392603      546374
315448      355047
1052591    1

In [43]:
model = Lasso(max_iter=1e4) #分析了训练数据，存在大量共线，可使用L1正则化消除共线
model.fit(x_train, y_train)
print(model.coef_)
print(len(model.coef_))

[ 1.69341883e-07  5.63455158e-06 -5.59014840e-07  2.44806255e-06
  1.29271159e-05 -2.02717754e-05 -6.31147203e-07  3.66459482e-07
  9.12525177e-07 -1.93078141e-06 -4.23377641e-03  6.39385633e-03
  1.56027300e-02 -1.13214028e-02 -7.95784079e-03  7.82507152e-03
 -5.59751729e-03  3.83482384e-03 -5.79091531e-03  4.31236702e-04
  4.49666627e-03 -4.93278489e-03 -2.88068437e-03  4.35721150e-04
  4.99747960e-04 -9.17476556e-03 -3.69248982e-03  2.78655212e-03
  9.36521834e-03 -1.09089228e-02  1.14110805e-02 -7.56458929e-02
  0.00000000e+00 -0.00000000e+00 -1.12155864e-02 -1.71158658e-01
 -1.12695791e-01  4.08690469e-01 -3.03881527e-01 -2.00115946e+00
  0.00000000e+00  6.71943032e-01 -2.25896555e-01  2.50901023e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.12893457e-01
  0.00000000e+00 -0.00000000e+00 -2.54249040e+00 -3.06173804e+00
 -6.86526687e+00  8.00667868e-01  0.00000000e+00  1.99301728e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [33]:
l_pred=model.predict(X2)
np.sqrt(mean_squared_error(y1,l_pred))

402.36355015764485

In [63]:
la_coef=pd.DataFrame(model.coef_,x_train.columns,columns=['coef'])
la_coef[abs(la_coef.values) < 1e-6]

,coef
wood_add_value,1.693419e-07
stone_add_value,-5.590148e-07
meat_add_value,-6.311472e-07
meat_reduce_value,3.664595e-07
magic_add_value,9.125252e-07
bd_training_hut_level,0.000000e+00
bd_healing_lodge_level,-0.000000e+00
bd_warehouse_level,0.000000e+00
bd_market_level,-0.000000e+00
bd_hero_gacha_level,-0.000000e+00


In [75]:
# model2 = Lasso() #分析了训练数据，存在大量共线，可使用L1正则化消除共线
# model2.fit(X, y)
# print(model2.coef_)
# print(len(model2.coef_))

# l_pred2=model2.predict(X2)
# np.sqrt(mean_squared_error(y1,l_pred2))

# la_coef2=pd.DataFrame(model2.coef_,X.columns,columns=['coef'])
# la_coef2[abs(la_coef2.values) < 1e-6]


# la_used_col=la_coef[abs(la_coef['coef'])< 1e-06]
# la_used_col2=la_coef2[abs(la_coef2['coef'])< 1e-06]

# la_used_col.shape

# la_used_col2.shape

# train_7pay=data[data['pay_price']>0].copy()
# train_7pay=train_7pay.drop(['register_time'],axis=1)
# # test_7pay=data_test2[data_test2['pay_price']>0].copy()

# # train_7pay2=data_train2[data_train2['pay_price']>0].copy()
# # test_7pay2=data_test2[data_test2['pay_price']>0].copy()
# train_7pay_y=train_7pay.prediction_pay_price
# train_7pay_X=train_7pay.drop(labels=['prediction_pay_price','user_id'],axis=1)
# train_7pay_user_id=train_7pay.user_id
# # test_7pay_X=test_7pay.drop(labels=['user_id'],axis=1)
# # test_7pay_user_id=test_7pay.user_id

# drop_col=[i for i in train_7pay.columns.values if 'prediction' in  i]
# drop_col.append('user_id')
# X=train_7pay.drop(labels=drop_col,axis=1)
# y=train_7pay.prediction_pay_price
# # X=train_7pay_X
# y=train_7pay_y
# X.replace([np.inf,-np.inf],np.nan,inplace=True)
# X.fillna(0.0,inplace=True)

# X['user_id']=train_7pay_user_id

# X1,val_X1,y1,val_y1=train_test_split(X,y,test_size=0.2,random_state=17)
# X2=X1.drop(labels='user_id',axis=1)
# val_X2=val_X1.drop(labels='user_id',axis=1)
# X2_user_id=X1.user_id
# val_X2_user_id=val_X1.user_id

# X.drop(labels='user_id',axis=1,inplace=True)

# x_train_new1=x_train.drop(labels=la_used_col.index,axis=1)
# x_train_new2=x_train.drop(labels=la_used_col2.index,axis=1)

# X_new1=X.drop(labels=la_used_col.index,axis=1)
# X_new2=X.drop(labels=la_used_col2.index,axis=1)

# X2_new1=X2.drop(labels=la_used_col.index,axis=1)
# X2_new2=X2.drop(labels=la_used_col2.index,axis=1)

# val_X2_new1=val_X2.drop(labels=la_used_col.index,axis=1)
# val_X2_new2=val_X2.drop(labels=la_used_col2.index,axis=1)


# model3 = Lasso() #分析了训练数据，存在大量共线，可使用L1正则化消除共线
# model3.fit(x_train_new1, y_train)
# l_pred3=model3.predict(X2_new1)
# np.sqrt(mean_squared_error(y1,l_pred3))




# model4 = Lasso() #分析了训练数据，存在大量共线，可使用L1正则化消除共线
# model4.fit(x_train_new2, y_train)
# l_pred4=model4.predict(X2_new2)
# np.sqrt(mean_squared_error(y1,l_pred4))

In [96]:
none_mean=la_coef[abs(la_coef.values) < 1e-6].index
x_train_final=x_train.drop(none_mean,axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(x_train_final,y_train,test_size=0.2,random_state=0)

In [77]:
X_train.shape

(220573, 50)

In [98]:
X_test.head()

,wood_reduce_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,magic_reduce_value,infantry_add_value,infantry_reduce_value,cavalry_add_value,cavalry_reduce_value,shaman_add_value,...,sr_infantry_atk_level,sr_shaman_atk_level,sr_construction_speed_level,sr_outpost_durability_level,pvp_battle_count,pvp_win_count,pve_win_count,avg_online_minutes,pay_price,pay_count
923466,2240882.0,542944.0,0.0,0.0,10500.0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.166668,0.0,0
2043512,4372092.0,1061584.0,192000.0,0.0,351434.0,465,2280,2515,2125,2545,...,0,0,2,1,1,0,25,162.666672,0.0,0
151362,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,12.666667,0.0,0
1251705,81797.0,0.0,0.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,1,73.833336,0.0,0
132999,1002005.0,391296.0,0.0,0.0,68390.0,234,9,457,7,225,...,0,0,0,0,0,0,6,33.333332,0.0,0


In [99]:
X_test_7pays = X_test['pay_price']

In [100]:
scaler=StandardScaler(with_mean=False).fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [101]:
Y_train.shape
type(Y_train)

pandas.core.frame.DataFrame

In [84]:
SGDModel=GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
SGDModel.fit(X_train,Y_train.values.ravel())


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [86]:
import datetime
nowTime=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model_name='SGDModel'
name='tf_'+model_name+'_'+str(nowTime)+'.gz'
# result.to_csv("F:\\study\\ml\\DataSet\\Tap4fun\\SubMission\\"+name+'.csv',index=False,encoding='utf-8')

joblib.dump(value=SGDModel,filename="F:\\study\\ml\\DataSet\\Tap4fun\\Model\\"+name,compress=True)

['F:\\study\\ml\\DataSet\\Tap4fun\\Model\\tf_SGDModel_20180927100046.gz']

In [102]:
y_pred=SGDModel.predict(X_test)
np.sqrt(mean_squared_error(Y_test,y_pred))

200.9465959638839

In [105]:
X_test_7pays.head()

923466     0.0
2043512    0.0
151362     0.0
1251705    0.0
132999     0.0
Name: pay_price, dtype: float32

In [106]:
y_pred2

array([1.31146475, 2.12787348, 1.31146475, ..., 1.31146475, 1.31146475,
       1.31146475])

In [109]:
y_pred2[y_pred2< X_test_7pays]

array([   2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.92193336,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,   26.47245862,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.92193336,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.96753737,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912864,
          2.82912864,  -16.79536975,    2.82912864,    2.82912864,
          2.82912864,    2.96753737,    2.82912864,    2.82912864,
          2.82912864,    2.82912864,    2.82912864,    2.82912

In [111]:
y_pred2=pd.DataFrame(list(zip(X_test_7pays,y_pred)),columns=['pay_price','pred_price'])


In [113]:
y_pred2[y_pred2.pred_price< y_pred2.pay_price].head()

,pay_price,pred_price
473,2.98,2.829129
957,2.97,2.829129
1997,2.97,2.829129
1998,2.98,2.829129
2017,2.98,2.829129


In [121]:
y_pred2.loc[y_pred2.pred_price< y_pred2.pay_price,'pred_price']=y_pred2[y_pred2.pred_price< y_pred2.pay_price]['pay_price']

In [123]:
y_pred2.iloc[2017,:]

pay_price     2.98
pred_price    2.98
Name: 2017, dtype: float64

In [124]:
y_pred2.loc[y_pred2.pred_price<0,'pred_price']=0

In [125]:
y_pred2.loc[(y_pred2.pred_price>0) & (y_pred2.pred_price<1.4)].head()

,pay_price,pred_price
0,0.0,1.311465
2,0.0,1.311465
3,0.0,1.311465
4,0.0,1.311465
5,0.0,1.311465


In [126]:
y_pred2.loc[(y_pred2.pred_price>0) & (y_pred2.pred_price<1.4),'pred_price']=0.99

In [127]:
y_pred2.iloc[0,:]

pay_price     0.00
pred_price    0.99
Name: 0, dtype: float64

In [147]:
y_pred22=SGDModel.predict(X_train)

In [148]:
np.sqrt(mean_squared_error(Y_train,y_pred22))

114.65528315842384

In [129]:
np.sqrt(mean_squared_error(Y_test,y_pred2.pred_price))

200.94246696147485

In [130]:
r2_score(Y_test,y_pred2.pred_price)

0.3702266972833469

In [131]:
r2_score(Y_test,y_pred)

0.37020081562438745

In [134]:
train_7pay=data[data['pay_price']>0].copy()
train_7pay=train_7pay.drop(['register_time'],axis=1)
# test_7pay=data_test2[data_test2['pay_price']>0].copy()

# train_7pay2=data_train2[data_train2['pay_price']>0].copy()
# test_7pay2=data_test2[data_test2['pay_price']>0].copy()
train_7pay_y=train_7pay.prediction_pay_price
train_7pay_X=train_7pay.drop(labels=['prediction_pay_price','user_id'],axis=1)
train_7pay_user_id=train_7pay.user_id
# test_7pay_X=test_7pay.drop(labels=['user_id'],axis=1)
# test_7pay_user_id=test_7pay.user_id

drop_col=[i for i in train_7pay.columns.values if 'prediction' in  i]
drop_col.append('user_id')
X=train_7pay.drop(labels=drop_col,axis=1)
y=train_7pay.prediction_pay_price
# X=train_7pay_X
y=train_7pay_y
X.replace([np.inf,-np.inf],np.nan,inplace=True)
X.fillna(0.0,inplace=True)

X['user_id']=train_7pay_user_id

X1,val_X1,y1,val_y1=train_test_split(X,y,test_size=0.1,random_state=17)
X2=X1.drop(labels='user_id',axis=1)
val_X2=val_X1.drop(labels='user_id',axis=1)
X2_user_id=X1.user_id
val_X2_user_id=val_X1.user_id

X.drop(labels='user_id',axis=1,inplace=True)

x_train_new1=x_train.drop(labels=none_mean,axis=1)
X_new1=X.drop(labels=none_mean,axis=1)
X2_new1=X2.drop(labels=none_mean,axis=1)
val_X2_new1=val_X2.drop(labels=none_mean,axis=1)

In [167]:
y_pred3=SGDModel.predict(X2_new1)
np.sqrt(mean_squared_error(y1,y_pred3))

16967.238540839175

In [139]:
Y_train.shape

(220573, 1)

In [138]:
Y_test.shape

(55144, 1)

In [143]:
SGDModel2=GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
SGDModel2.fit(X2_new1,y1.values.ravel())

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [153]:
y_pred24=SGDModel2.predict(val_X2_new1)
np.sqrt(mean_squared_error(val_y1,y_pred24))

593.7399450077135

In [154]:
r2_score(val_y1,y_pred24)

0.07426589297018349

In [140]:
data_test=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [150]:
data1=data_test.copy()
data1.shape

(828934, 108)

In [163]:
data_test1=data1[data1.avg_online_minutes < 12]
data_test2=data1[data1.avg_online_minutes >= 12]
data_test2_7pays=data_test2['pay_price']

In [165]:
data_test_part1=data_test1[['user_id','pay_price']].copy()
data_test_part1.rename(columns={'pay_price':'predictin_pay_price'},inplace=True)
data_test_part1.to_csv("F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part1.csv")

In [166]:
data_test2_id=data_test2['user_id'].values
data_test2_id_final=pd.DataFrame(data_test2_id,columns=['user_id'])
data_test2=data_test2.drop(['user_id','register_time'],axis=1)

In [169]:
data_test2_final=data_test2.drop(none_mean,axis=1)
data_test2_final=scaler.transform(data_test2_final.values)
data_test2_pred=SGDModel.predict(data_test2_final)

In [170]:
test2_pred=pd.DataFrame(list(zip(data_test2_id,data_test2_pred)),columns=['user_id','prediction_pay_price'])

In [171]:
test2_pred['pay_price']=data_test2.pay_price

In [173]:
test2_pred.loc[test2_pred.prediction_pay_price < test2_pred.pay_price].head()

,user_id,prediction_pay_price,pay_price
126,15761,1.311465,5.980000
309,18458,1.311465,16.959999
470,19804,2.829129,93.900002
483,19910,1.311465,10.980000
558,20339,1.311465,9.940000


In [175]:
test2_pred.loc[test2_pred.prediction_pay_price < test2_pred.pay_price,'prediction_pay_price']=\
test2_pred.loc[test2_pred.prediction_pay_price < test2_pred.pay_price,'pay_price']

In [176]:
test2_pred.loc[126,:]

user_id                 15761.00
prediction_pay_price        5.98
pay_price                   5.98
Name: 126, dtype: float64

In [177]:
test2_pred.loc[test2_pred.prediction_pay_price < 0,'prediction_pay_price']=0
test2_pred.loc[(test2_pred.prediction_pay_price > 0) & (test2_pred.prediction_pay_price < 1.4) ,'prediction_pay_price']=0.99

In [179]:
result=pd.DataFrame()
result['user_id']=data1.user_id
result['prediction_pay_price']=0.0

In [194]:
result=result.merge(test2_pred,how='left',on='user_id')

In [195]:
result.fillna(0.0,inplace=True)

In [196]:
result

,user_id,prediction_pay_price_x,prediction_pay_price_y,pay_price
0,14933,0.0,0.000000,0.0
1,14934,0.0,0.000000,0.0
2,14935,0.0,0.990000,0.0
3,14936,0.0,0.000000,0.0
4,14937,0.0,0.000000,0.0
5,14938,0.0,0.000000,0.0
6,14939,0.0,0.990000,0.0
7,14940,0.0,0.000000,0.0
8,14941,0.0,0.000000,0.0
9,14942,0.0,0.000000,0.0


In [197]:
result.rename(columns={'prediction_pay_price_y':"prediction_pay_price"},inplace=True)

In [198]:
result.head()

,user_id,prediction_pay_price_x,prediction_pay_price,pay_price
0,14933,0.0,0.00,0.0
1,14934,0.0,0.00,0.0
2,14935,0.0,0.99,0.0
3,14936,0.0,0.00,0.0
4,14937,0.0,0.00,0.0


In [199]:
result.drop(labels=['prediction_pay_price_x','pay_price'],axis=1,inplace=True)

In [201]:
result.head(30)

,user_id,prediction_pay_price
0,14933,0.000000
1,14934,0.000000
2,14935,0.990000
3,14936,0.000000
4,14937,0.000000
5,14938,0.000000
6,14939,0.990000
7,14940,0.000000
8,14941,0.000000
9,14942,0.000000


In [202]:
import datetime
nowTime=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
name='tf_'+str(nowTime)
result.to_csv("F:\\study\\ml\\DataSet\\Tap4fun\\SubMission\\"+name+'.csv',index=False,encoding='utf-8')
